In [70]:
from spheres import *
from pytket.backends.ibm import AerBackend, AerStateBackend, IBMQBackend
from pytket.qiskit import tk_to_qiskit

n_qubits = 1
depth = 4 
n_shots = 8000 

n_copies = 2

every = 2
pairwise = True
reuse_cntrls = False

latex = False
use_remote_simulator = False

noise_model_from = "ibmq_santiago"

################################################################################################

circ_specification = random_circuit(n_qubits=n_qubits, depth=depth)
circ = build_circuit(circ_specification)
if latex:
    circ.to_latex_file("circ.tex")

################################################################################################

sym_circ_info = symmetrize_circuit(circ_specification,
                                   n_copies=n_copies,
                                   every=every, 
                                   pairwise=pairwise,
                                   reuse_cntrls=reuse_cntrls)
sym_circ = sym_circ_info["circuit"]
if latex:
    sym_circ.to_latex_file("sym_circ.tex")

total_sym_qubits = len(sym_circ.qubits)
print("%d total qubits for symmetrized circuit" % total_sym_qubits)

################################################################################################

analytic_circ = circ.copy()
state_backend = AerStateBackend()
state_backend.compile_circuit(analytic_circ)
circ_distribution = state_backend.get_result(state_backend.process_circuit(analytic_circ)).get_distribution()

print("analytical dist for original circuit:")
for bitstr, prob in circ_distribution.items():
    print("  %s: %f" % ("".join([str(b) for b in bitstr]), prob))

5 total qubits for symmetrized circuit
analytical dist for original circuit:
  0: 0.853553
  1: 0.146447


In [71]:
from qiskit import QuantumCircuit, QuantumRegister, Aer, execute, IBMQ, transpile
from qiskit.providers.ibmq.managed import IBMQJobManager
from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel
provider = IBMQ.load_account()
job_manager = IBMQJobManager()

def error_calibration(n_qubits, noise_model, use_remote_simulator=False):
    meas_calibs, state_labels = complete_meas_cal(qr=QuantumRegister(n_qubits), circlabel='mcal')
    if not use_remote_simulator:
        noise_job = execute(meas_calibs, backend=Aer.get_backend("qasm_simulator"), shots=n_shots, noise_model=noise_model)
        cal_results = noise_job.result()
    else:
        noise_job = job_manager.run(meas_calibs, backend=provider.get_backend("ibmq_qasm_simulator"), shots=n_shots, noise_model=noise_model)
        cal_results = noise_job.results().combine_results()                                  
    meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='mcal')
    return meas_fitter.filter

noise_model = NoiseModel.from_backend(IBMQ.providers()[0].get_backend(noise_model_from))

ibmqfactory.load_account:WARNING:2020-11-22 22:20:57,811: Credentials are already in use. The existing account in the session will be replaced.


In [73]:
noisy_circ = circ.copy().measure_all()
noisy_backend = AerBackend(noise_model)
noisy_backend.compile_circuit(noisy_circ)
noisy_circ_result = noisy_backend.get_result(noisy_backend.process_circuit(noisy_circ, n_shots=n_shots))

print("noisy dist for original circuit:")
for bitstr, prob in noisy_circ_result.get_distribution().items():
    print("  %s: %f" % ("".join([str(b) for b in bitstr]), prob))

noisy dist for original circuit:
  0: 0.847375
  1: 0.152625


In [76]:
circ_meas_filter = error_calibration(n_qubits, noise_model)
noisy_circ_counts = dict([("".join([str(b) for b in bitstr]), count) for bitstr, count in noisy_circ_result.get_counts().items()])
mitigated_circ_counts = circ_meas_filter.apply(noisy_circ_counts)

print("mitigated dist for original circuit:")
for bitstr, prob in probs_from_counts(mitigated_circ_counts).items():
    print("  %s: %f" % (bitstr, prob))

mitigated dist for original circuit:
  0: 0.857310
  1: 0.142690


In [77]:
def display_sym_dists(sym_dists):
    exp_dists, avg_dist = sym_dists["exp_dists"], sym_dists["avg_dist"], 
    print("  postselected dists:")
    for i, dist in enumerate(exp_dists):
        print("    experiment %d:" % i)
        for bitstr, prob in dist.items():
            print("      %s: %f" % ("".join([str(b) for b in bitstr]), prob))
    print("  averaged dists:")
    for bitstr, prob in avg_dist.items():
        print("    %s: %f" % ("".join([str(b) for b in bitstr]), prob)) 

def qiskit_pytket_counts(counts):
    return dict([(tuple([int(b) for b in "".join(bitstr.split())][::-1]), count) for bitstr, count in counts.items()])

In [78]:
clean_sym_circ = sym_circ.copy()

clean_backend = AerBackend()
clean_backend.compile_circuit(clean_sym_circ)

if not use_remote_simulator:
    clean_sym_circ_counts = clean_backend.get_result(clean_backend.process_circuit(clean_sym_circ, n_shots=n_shots)).get_counts()
else:
    qis_clean_sym_circ = tk_to_qiskit(clean_sym_circ)
    qis_clean_sym_circ_counts = execute(qis_clean_sym_circ, backend=provider.get_backend("ibmq_qasm_simulator"), shots=8000).result().get_counts()
    clean_sym_circ_counts = qiskit_pytket_counts(qis_clean_sym_circ_counts)

print("clean sym circ dists:")
display_sym_dists(process_sym_counts(sym_circ_info, clean_sym_circ_counts))

clean sym circ dists:
  postselected dists:
    experiment 0:
      0: 0.851375
      1: 0.148625
    experiment 1:
      0: 0.855000
      1: 0.145000
  averaged dists:
    0: 0.853187
    1: 0.146813


In [79]:
noisy_sym_circ = sym_circ.copy()
noisy_backend.compile_circuit(noisy_sym_circ)

if not use_remote_simulator:
    noisy_sym_circ_counts = noisy_backend.get_result(noisy_backend.process_circuit(noisy_sym_circ, n_shots=n_shots)).get_counts()
else:
    qis_noisy_sym_circ = tk_to_qiskit(noisy_sym_circ)
    qis_noisy_sym_circ_counts = execute(qis_noisy_sym_circ, backend=provider.get_backend("ibmq_qasm_simulator"), shots=8000, noise_model=noise_model).result().get_counts()
    noisy_sym_circ_counts = qiskit_pytket_counts(qis_noisy_sym_circ_counts)

print("noisy sym circ dists:")
display_sym_dists(process_sym_counts(sym_circ_info, noisy_sym_circ_counts))

noisy sym circ dists:
  postselected dists:
    experiment 0:
      0: 0.804554
      1: 0.195446
    experiment 1:
      0: 0.798861
      1: 0.201139
  averaged dists:
    0: 0.801708
    1: 0.198292


In [80]:
sym_circ_meas_filter = error_calibration(total_sym_qubits, noise_model, use_remote_simulator=use_remote_simulator)
noisy_sym_circ_counts = dict([("".join([str(b) for b in bitstr]), count) for bitstr, count in noisy_sym_circ_counts.items()])
mitigated_sym_circ_counts = sym_circ_meas_filter.apply(noisy_sym_circ_counts)
mitigated_sym_circ_counts = dict([(tuple([int(b) for b in bitstr]), count) for bitstr, count in mitigated_sym_circ_counts.items()])
mitigated_sym_dists = process_sym_counts(sym_circ_info, mitigated_sym_circ_counts)

print("mitigated sym circ dists:")
display_sym_dists(mitigated_sym_dists)

mitigated sym circ dists:
  postselected dists:
    experiment 0:
      0: 0.808392
      1: 0.191608
    experiment 1:
      0: 0.804018
      1: 0.195982
  averaged dists:
    0: 0.806205
    1: 0.193795


In [81]:
basis = list(product([0,1], repeat=n_qubits))
analytic_circ_probs = []
mitigated_sym_circ_probs = []
for b in basis:
    if b in circ_distribution:
        analytic_circ_probs.append(circ_distribution[b])
    else:
        analytic_circ_probs.append(0)
    if b in mitigated_sym_dists["avg_dist"]:
        mitigated_sym_circ_probs.append(mitigated_sym_dists["avg_dist"][b])
    else:
        mitigated_sym_circ_prob.append(0)

print("desired results: %s" % analytic_circ_probs)
print("actual results: %s" % mitigated_sym_circ_probs)

error = np.linalg.norm(np.array(mitigated_sym_circ_probs) - np.array(analytic_circ_probs))
print("error: %f" % error)

desired results: [0.853553390593274, 0.14644660940672607]
actual results: [0.8062050577421851, 0.19379494225781496]
error: 0.066961
